In [115]:
import pysftp
import os 
import numpy as np
import pandas as pd
import netCDF4
import numpy as np

from scipy import io
from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [88]:
path_to_folder='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/TEMPERATURE/HOURLY/'

list_ = os.listdir(path=path_to_folder)
list_.sort()
print(len(list_))

number_days = len(list_)//12
for i in range(number_days):
    files_day = (list_[1+i*12:i*12 + 12])


In [105]:
number_days = len(list_)//12
files_list = []
for i in range(number_days):
    files_day = (list_[1+i*12:i*12 + 12])
    files_list.append(files_day)

In [112]:
result = []
for sublist in files_list:
    filtered_sublist = [filename for filename in sublist if filename.startswith('2018')]
    if filtered_sublist:
        result.append(filtered_sublist)

In [128]:
grid = map_grid(grid_size=None, source = 'era5')
wizard = data_wizard()
res_mean = []
res_mean_selection = []
file_names = []


path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/WIND_EASTWARD/HOURLY/'
data_name = 'wind_eastward'
geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180}

for day in result:
    # create a list to hold the dataframes
    array_list = []
    for file in day:
        data = np.load(path + file)['arr_0']
        array_list.append(data)
        data = np.mean(array_list, axis=0)
        
    select_area_indexer = grid.select_area_indexer(
    min_lat= geo_filtering['min_lat'],
    max_lat= geo_filtering['max_lat'],
    min_lon= geo_filtering['min_lon'],
    max_lon= geo_filtering['max_lat'])

    data_selection = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data)

    file_names.append(file[:10])
    res_mean.append(np.nanmean(data))
    res_mean_selection.append(np.nanmean(data_selection))

    df = pd.DataFrame(data={
    'dates' : file_names,
    f'{data_name}_mean' : res_mean,
    f'{data_name}_mean_selection' : res_mean_selection
    })

    df['dates'] = pd.to_datetime(df['dates'])
    df = df.sort_values(by='dates')
    df = df.reset_index()
    df = df.drop(columns=['index'])
 

In [129]:
import plotly.graph_objects as go

# df = df_dataset
# Create figure and plot the DataFrame
fig = go.Figure()

# fig.add_trace(go.Scatter(x=df['dates'], y=df['res_filtered'].rolling(3).mean(), name='Res Filtered', mode='markers', marker=dict(color='blue', size=2)))
fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_eastward_mean_selection'], name='wind_eastward_mean_selection', line=dict(color='blue')))

# Add a dashed line at y=0
fig.add_shape(type='line', x0=df['dates'].min(), y0=0, x1=df['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-50, 80],
        title='wind_eastward_mean_selection'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude in',
    legend=dict(orientation="h", y=1.2, x=0.5)
)

# Show the plot
fig.show()